In [12]:
import numpy as np
import pandas as pd
import pickle
import nltk

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [4]:
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

from gensim import corpora, models, similarities, matutils

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [13]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/jess/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jess/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import linguistica as lxa
from collections import Counter

In [9]:
en_df = pickle.load(open("en_df.pkl", "rb"))

In [14]:
en_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to,text,lang
0,1244776419676303365,1244776419676303365,2020-03-30 18:59:59 CDT,2020-03-30,18:59:59,-600,1431335726,goreactforasl,GoReact for ASL,NaN,...,"['signlanguage', 'deaf']",[],https://twitter.com/GoReactforASL/status/12447...,False,NaN,0,NaN,[],Health Workers Study SignLanguage to Care for...,en
119129,1244701870326571013,1244701870326571013,2020-03-30 14:03:45 CDT,2020-03-30,14:03:45,-600,907496840,drmanishshukla_,Dr Manish Shukla,NaN,...,"['nizamuddin', 'coronaupdatesinindia']",[],https://twitter.com/drmanishshukla_/status/124...,False,NaN,0,NaN,[],Corona Virus has spread among some of those wh...,en
119130,1244701869857034248,1244701449700937734,2020-03-30 14:03:45 CDT,2020-03-30,14:03:45,-600,1006650378254012418,sheagrandequeen,katie,NaN,...,[],[],https://twitter.com/sheagrandequeen/status/124...,False,NaN,0,NaN,"[{'screen_name': 'ArianaGrande', 'name': 'Aria...",ArianaGrande bro pls after corona have this...,en
119131,1244701869777117188,1244679079443357696,2020-03-30 14:03:45 CDT,2020-03-30,14:03:45,-600,1100443875531837440,betaal56,Storyteller,NaN,...,[],[],https://twitter.com/betaal56/status/1244701869...,False,NaN,0,NaN,"[{'screen_name': 'dhanyarajendran', 'name': 'D...",dhanyarajendran Thid Tabliqi thing has become...,en
119133,1244701868502237190,1244701868502237190,2020-03-30 14:03:45 CDT,2020-03-30,14:03:45,-600,509512845,edward_a123,Eddy,NaN,...,[],[],https://twitter.com/Edward_A123/status/1244701...,False,NaN,0,NaN,[],Yo I can’t wait until self quarantine ends and...,en


In [15]:
en_df['tokens'] = en_df['text'].apply(lambda x: word_tokenize(x))

In [20]:
en_df['tokens'][0:5]

0         [Health, Workers, Study, SignLanguage, to, Car...
119129    [Corona, Virus, has, spread, among, some, of, ...
119130    [ArianaGrande, bro, pls, after, corona, have, ...
119131    [dhanyarajendran, Thid, Tabliqi, thing, has, b...
119133    [Yo, I, can, ’, t, wait, until, self, quaranti...
Name: tokens, dtype: object

In [30]:
stop_words_standard = stopwords.words('english')
stop_words_long = open('/Users/jess/workspace/Metis_Projects/Project_4/stopwords/stopwords_long_ranks_nl.txt', 'r', encoding='utf-8').read().split('\n')
stop = set(stop_words_standard + stop_words_long + ['https' , 'http'])

In [19]:
en_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'reply_to', 'text', 'lang', 'tokens'],
      dtype='object')

In [21]:
en_df['filtered'] = en_df['tokens'].apply(lambda tweet: [word for word in tweet if word not in stop])

In [22]:
en_df['filtered'].iloc[0]

['Health',
 'Workers',
 'Study',
 'SignLanguage',
 'Care',
 'Deaf',
 'COVID',
 'Patients',
 'https',
 'imoFWhCBBg']

In [18]:
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jess/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [23]:
def nouns(text):
    is_noun = lambda pos: pos[:2] in ('NN', 'NNS')
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [26]:
en_df['nouns'] = en_df['text'].apply(nouns)

In [ ]:
# ^ last cell run

In [27]:
en_df['filtered_nouns'] = en_df['nouns'].apply(lambda tweet: [word for word in tweet if word not in stop])

In [33]:
en_df[['tokens', 'filtered', 'nouns','filtered_nouns']].head()

,tokens,filtered,nouns,filtered_nouns
0,"[Health, Workers, Study, SignLanguage, to, Car...","[Health, Workers, Study, SignLanguage, Care, D...",Health Workers Study SignLanguage Care Deaf CO...,"[H, , W, , S, , S, L, , C, , D, , C, O, ..."
119129,"[Corona, Virus, has, spread, among, some, of, ...","[Corona, Virus, spread, attended, religious, p...",Corona Virus prayer meeting March Markaz Nizam...,"[C, , V, , , , M, , M, , N, , , T, , ..."
119130,"[ArianaGrande, bro, pls, after, corona, have, ...","[ArianaGrande, bro, pls, corona, pls, thx]",ArianaGrande bro pls corona look pls thx,"[A, G, , , , , , ]"
119131,"[dhanyarajendran, Thid, Tabliqi, thing, has, b...","[dhanyarajendran, Thid, Tabliqi, thing, bigges...",dhanyarajendran Thid Tabliqi thing community s...,"[ , T, , T, , , , , , , , , , C, , ]"
119133,"[Yo, I, can, ’, t, wait, until, self, quaranti...","[Yo, I, ’, wait, quarantine, ends, corona, virus]",Yo wait ends corona virus,"[Y, , , , ]"


In [34]:
pickle.dump(en_df, open('en_df.pkl', 'wb'))

In [ ]:
vectorizer = TfidfVectorizer(min_df=2,
                            stop_words=my_stop_words)
tfidf_en = vectorizer.fit_transform(en_df['filtered'])

In [ ]:
pickle.dump(tfidf_en, open('tfidf_en_sparse.pkl', 'wb'))
pickle.dump(vectorizer, open('en_vectorizer.pkl', 'wb'))

In [ ]:
def get_ngrams(df):
    data_dict = dict()

    tokens = []
    for row in en_df.index.tolist():
        tokens += en_df.loc[row, 'filtered'].split()

    lxa_object = lxa.from_corpus(tokens)

    unigrams = lxa_object.word_unigram_counter()
    bigrams = lxa_object.word_bigram_counter()
    trigrams = lxa_object.word_trigram_counter()

    df_unigrams = pd.DataFrame.from_dict(unigrams, orient='index') \
    .reset_index()


    df_bigrams = pd.DataFrame.from_dict(bigrams, orient='index') \
    .reset_index()


    df_trigrams = pd.DataFrame.from_dict(trigrams, orient='index') \
    .reset_index()

    df_unigrams.columns = ['N-Gram','Frequency']
    df_bigrams.columns = ['N-Gram','Frequency']
    df_trigrams.columns = ['N-Gram', 'Frequency']
    
    df_bigrams[['word1', 'word2']] = df_bigrams['N-Gram'].apply(pd.Series)
    df_trigrams[['word1', 'word2', 'word3']] = df_trigrams['N-Gram'].apply(pd.Series)

    data_dict['Unigrams'] = df_unigrams
    data_dict['Bigrams'] = df_bigrams
    data_dict['Trigrams'] = df_trigrams
    
    return data_dict

In [ ]:
df_unigrams.columns = ['N-Gram','Frequency']
df_bigrams.columns = ['N-Gram','Frequency']
df_trigrams.columns = ['N-Gram', 'Frequency']

In [ ]:
df_bigrams[['word1', 'word2']] = df_bigrams['N-Gram'].apply(pd.Series)
df_trigrams[['word1', 'word2', 'word3']] = df_trigrams['N-Gram'].apply(pd.Series)


In [ ]:
data_dict['Unigrams'] = df_unigrams
data_dict['Bigrams'] = df_bigrams
data_dict['Trigrams'] = df_trigrams

data_dict.keys()

In [ ]:
def viz(chart_data, num_grams=30, ascending=False, font_scale=1, 
        chart_scale='poster', chart_size=15, save_to_file=True, title=""):
   
    chart_data = chart_data.head(num_grams)\
    .reset_index().sort_values(by='Frequency', ascending=ascending).copy()
    
    sns.set_context(chart_scale, font_scale=font_scale)
    
    sns_plot = sns.catplot(x="Frequency", y='N-Gram', data=chart_data, kind="bar",\
                           size=chart_size, palette="flare");
    plt.title(title)
    plt.show();
    if save_to_file is not False:
        sns_plot.savefig(table_name)
    

In [ ]:
titles = ['Unigrams', 'Bigrams', 'Trigrams']

for title in titles:
    viz(data_dict[title].sort_values(by='Frequency', ascending=False), num_grams=10, \
       chart_scale='poster', chart_size=9, title=title)